In [1]:
%load_ext line_profiler
%load_ext cython
import line_profiler
import random
import array
import numpy as np


In [2]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1
import cython
@cython.linetrace(True)
@cython.binding(True)
def query_integral_image(unsigned int[:,:] integral_image, int size_x, int
                         size_y, random_state):
    cdef int x = integral_image.shape[0]
    cdef int y = integral_image.shape[1]
    cdef int area, i, j
    cdef int hits = 0

    # count how many possible locations
    for i in xrange(x - size_x):
        for j in xrange(y - size_y):
            area = integral_image[i, j] + integral_image[i + size_x, j + size_y]
            area -= integral_image[i + size_x, j] + integral_image[i, j + size_y]
            if not area:
                hits += 1
    if not hits:
        # no room left
        return None
    # pick a location at random
    cdef int goal = random_state.randint(0, hits)
    hits = 0
    for i in xrange(x - size_x):
        for j in xrange(y - size_y):
            area = integral_image[i, j] + integral_image[i + size_x, j + size_y]
            area -= integral_image[i + size_x, j] + integral_image[i, j + size_y]
            if not area:
                hits += 1
                if hits == goal:
                    return i, j

In [3]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1
from cpython cimport array
import cython

@cython.linetrace(True)
@cython.binding(True)
def alternative(unsigned int[:, :] integral_image, int size_x, int size_y, random_state):
    cdef int hits = c_search_for_space(integral_image, size_x, size_y)
    if not hits:
        return None
    cdef int goal = random_state.randrange(1, hits + 1)
    cdef int coordinates_array[2]
    cdef int[:] coordinates_memory_view = coordinates_array
    c_iterate_for_coordinates(integral_image, size_x, size_y, goal, coordinates_memory_view)
    return coordinates_array

cdef int c_search_for_space(unsigned int[:, :] integral_image, int size_x, int size_y):
    cdef int x = integral_image.shape[0]
    cdef int y = integral_image.shape[1]
    cdef int area, i, j
    cdef int hits = 0
    for i in xrange(x - size_x):
        for j in xrange(y - size_y):
            area = integral_image[i, j] + integral_image[i + size_x, j + size_y]
            area -= integral_image[i + size_x, j] + integral_image[i, j + size_y]
            if not area:
                hits += 1
    return hits



cdef void c_iterate_for_coordinates(unsigned int[:, :] integral_image, int size_x, int size_y, int goal, int[:] coordinates_array):
    cdef int x = integral_image.shape[0]
    cdef int y = integral_image.shape[1]
    cdef int area, i, j
    cdef int hits = 0
    for i in xrange(x - size_x):
        for j in xrange(y - size_y):
            area = integral_image[i, j] + integral_image[i + size_x, j + size_y]
            area -= integral_image[i + size_x, j] + integral_image[i, j + size_y]
            if not area:
                hits += 1
            if hits == goal:
                coordinates_array[0] = i
                coordinates_array[1] = j
                return

In [4]:
# setting up objects
integral_image = np.zeros((400, 400), dtype=np.uint32)
size_x = 10
size_y = 10
random_state = random.Random()

In [5]:
# alternative is the modified function
%lprun -f query_integral_image -f alternative alternative(integral_image, size_x, size_y, random_state); query_integral_image(integral_image, size_x, size_y, random_state)

In [6]:
# timeit for original function

# Takes incredibly long to run so number is set to 1 for now in case someone runs the whole file
import timeit
number_of_timeit_runs = 1

timeit.timeit(number=number_of_timeit_runs,
              stmt="query_integral_image(integral_image, size_x, size_y, random_state)", 
              setup="from __main__ import query_integral_image, integral_image, size_x, size_y, random_state")

0.003906456730514794

In [7]:
# timeit for alternative
# timeit doesn't show the dramatic difference like in lprun
# my computer is really slow however

timeit.timeit(number=number_of_timeit_runs,
              stmt="alternative(integral_image, size_x, size_y, random_state)", 
              setup="from __main__ import alternative, integral_image, size_x, size_y, random_state")

0.0018121003338251729